In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
%load_ext google.colab.data_table

!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
import json
from bs4 import BeautifulSoup
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.use('ggplot')
mpl.rcParams.update({'font.size':16}) 
import matplotlib.ticker as mtick
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import bs4


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [3]:
url='https://it.m.wikipedia.org/wiki/Additivo_alimentare'
wd.get(url)
headers=[]
listas=[]
for num in range(4,12):
  headers=wd.find_element_by_class_name('mf-section-{}'.format(num))
  listas+=headers.find_elements_by_tag_name('ul')
# ingredientes=[header.find_elements_by_tag_name('ul') for header in headers]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  
/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:359: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


In [4]:
%cd /content/drive/MyDrive/files_foods

/content/drive/MyDrive/files_foods


In [5]:
data_raw={}
for nodo in listas:
  ing=nodo.find_elements_by_tag_name('li')
  for inner_nodo in ing:
    ref, *names=inner_nodo.text.split(' ')
    data_raw[ref]=' '.join(names)

/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:359: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


In [6]:
# from google.colab import files
# pd.Series(data_raw).to_excel('traduccion.xlsx')
# files.download('traduccion.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
df=pd.read_excel('traduccion_clean.xlsx', header=None, index_col=0)

In [12]:
df2=pd.read_excel('FOODs_Aditivos.xlsx')

In [29]:
borders=[n for n,row in df2.iterrows() if row.isna().all()] # Como cada ingrediente distinto está separado por una row de NaN's, busco las fronteras
null_row=df2.iloc[borders[0]].copy() # Genero una row vacía para insertar fronteras
reconstitutor_for_dataframe=[] # Contenedor para todas las rows modificadas
for n,node in enumerate(borders): # Se uitera sobre el index
  df_slice=df2.iloc[borders[n-1]+1:node,:] # El cachito de dataframe que representa un ingrediente.
  for n, row in df.iterrows(): # itero sobre el df que contiene los nombres traducidos.
    if row.name.lower() in ' '.join(df_slice["Unnamed: 0"].dropna().str.lower()): # el join contiene todos los nombres de los sinonimos, row.name es el nombre "de referencia"
      sel=df_slice.iloc[-1,:].copy() # Copio la ultima row para modificarla y apendarla al slice luego.
      for italian_name in row.dropna(): # Itero sobre todos los sinonimos traducidos
        sel['Unnamed: 0']=italian_name # Modifico el nombre de la ultima row con el sinonimo traducido
        df_slice=df_slice.append(sel) # se agrega la row al paquete
        break # ? 
  df_slice=df_slice.append(null_row) # Se agrega la null row al paquete
  reconstitutor_for_dataframe.append(df_slice) # se agrega el paquete al contenedor

In [15]:
pd.concat(reconstitutor_for_dataframe).reset_index(drop=True).to_excel('final_.xlsx')
files.download('final_.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>